# Computing mappability
Raffaele Fronza, Elvira Mingazova   
April, 2017
  
Plan:
- Key concepts of mappability (why is mappability important for the random model)  
- Mappability computing and visualization algorithm  
- Proving correctness of the results for the yest genome and human chrY  

## Key concepts of mappability  
*Based on the paper from Derrien T., Estellé J, Marco Sola S, et al. Fast computation and applications of genome mappability.  
https://www.ncbi.nlm.nih.gov/pubmed/22276185*

Genome sequencing technologies (HTS - high throughput screening) produce huge amounts of short sequences (reads), with different lengths, error rates etc. depending on the technology employed. After completing the sequencing itself those short reads need to be alligned (mapped) onto a reference genome. The genome however has a nonrandom nature and includes a significant proportion of repetitive sequences, so one needs to distinguish between uniquely mapping reads (can be aligned to only one single location) and multiply mapping reads (match multiple possible locations).

For a given genome, the proportion of uniquely mapped reads depends mostly on the length of the sequence reads produced by the experiment, and on the number of mismatches allowed during the mapping step.

Therefore, given the technical specifications of the sequencing experiment it is possible to compute a priori the mappability of the whole sequence, i.e., the inverse of the number of times that a read originating from any position in the reference genome maps to the genome itself – thus identifying, for instance, the regions that are truly “mappable”, that is those producing reads which map back unambiguously to themselves.  
  
Definition according to Derrien at. all:
Given some read length  k  , the    k-frequency Fk(x)    of a sequence at a given position  x  corresponds to the number of times the  k-mer  starting at position  x  appears in the sequence and in its reverse complement.

Mappability :
Mk(x) = 1/Fk(x)

m = number of mismatches allowed in an alignment

t = approximation parameter / threshold frequency





## Mappability computing and visualization algorithm  
  
http://wiki.bits.vib.be/index.php/Create_a_mappability_track  
  
Server: elvira@transposon  
Path to the genomes: /home/elvira/workspace/data/genomes  
Path to the tools: /home/elvira/workspace/bin  
  
**Example organism**: yeast
1. Download a genome of interest in fasta format  
  
    The yeast genome was downloaded from the UCSC server under http://hgdownload.soe.ucsc.edu/goldenPath/sacCer3/bigZips/. The filename for the complete genome sacCer3.2bit contains sequences for all 16 nuclear chromosomes and the mitochondrial genome.  
  
    The best way is to download the file directly from the terminal into the respective directory using the following bash command:  
    ``` bash
    wget http://hgdownload.soe.ucsc.edu/goldenPath/sacCer3/bigZips/sacCer3.2bit
    ```
  
    After that sacCer3.2bit needs to be converted to sacCer3.fa using twoBitToFa tool (downloaded from http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v287/):
    ``` bash
    twoBitToFa sacCer3.2bit SacCer3.fa
    ```  
2. Download GEM-tools  
  
    Tools were obtained and installed as described in http://algorithms.cnag.cat/wiki/GEM:Installation_instructions[7] (the core-2 version was used on the local machine)  
      
3. Create an indexed genome
  
    ``` bash
    pref="sacCer3"
    reference="sacCer3.fa"
    idxpref="sacCer3_index"
    thr=8;
    gem-indexer -T ${thr} -c dna -i ${reference} -o ${idxpref}
    ```
      
4. Compute mappability for several kmer lengths
  
    ``` bash
    pref="sacCer3"
    idxpref="sacCer3_index"
    thr=8;
 
    for kmer in $(seq 30 150 500); do
 
      # compute mappability data
      gem-mappability -T ${thr} -I ${idxpref}.gem -l ${kmer} -o ${pref}_${kmer}
 
      # convert results to wig and bigwig
      gem-2-wig -I ${idxpref}.gem -i ${pref}_${kmer}.mappability -o ${pref}_${kmer}
      wigToBigWig ${pref}_${kmer}.wig ${pref}_${kmer}.sizes ${pref}_${kmer}.bw
 
    done
    ```
5. Visualize the .bw files in IGV  
      
    Below is the screenshot of three mappability tracks for the whole yeast genome computed using kmer 30, 150 and 500 bp long. Above the maps there are greek numbers which represent respective chromosomes on the genome. The track called "Gene" represents gene density. By zooming in it is possible to look at the sequence of any chosen position. At a first glance the yeast genome seems to be of a high complexity since not so many unmappable regions are to be seen.  
      
    <img src="../images/yeastMappability.png">
      
    

## Correctness of the results  
Hier we want to prove manually at the different positions on the yeast genome whether the mappability score is correct. For that reason we will take a sequence of 30/150/500 bp from a specific position on the yeast genome and align it using blast, then we will compare the results with the map computed using GEM-tools.
  
#### Chromosome I, position 140008  
  
   <img src="../images/yeastchrI_YAL005C.png">  

According to the map above mappability for the yeast genome is 0.5 for kmer=30, 1 for kmer 150 and 0.5 for kmer 500.
Below is the sequence obtained from http://fungi.ensembl.org/Saccharomyces_cerevisiae/Gene/Sequence?db=core;g=YAL005C;r=I:139503-141431;t=YAL005C. It is the reverse strand, since the coding sequence is on the reverse strand. In order to find the sequence starting from 140008 and corresponding to the one seen under the map in IGV ("AAAGTGACTTCA...") we first need to convert this sequence below into the forward strand 

In [47]:
#chromosome:R64-1-1:I:138903:142031:-1
dna = """AATGTGAAAAATTTTTTCTTCAAAGGCTCGGTTGTCGACAAATTGTTACGTTGTGCTTTG
ATTTCTAAAGCGCTTCTTCACCTGCAGGTTCTGAGCCCTAAGAAAAAAAATTTCCTTGGT
TGAAAATGGCGGAAAAAAAAAATTCAGAAAAAGAAATAAAGCACGTGTGCGCGGTGTGTG
GATGATGGTTTCATCATTGTCAACGGCATTTTCGTTCTTGTGGATTGTTGTAAACTTTCC
AGAACATTCTAGAAAGAAAGCACACGGAACGTTTAGAAGCTGTCATTTGCGTTTTTTCTC
CAGATTTTAGTTGAGAAAGTAATTAAATTATTCTTCTTTTTCCAGAACGTTCCATCGGCG
GCAAAAGGGAGAGAAAGAACCCAAAAAGAAGGGGGGCCATTTAGATTAGCTGATCGTTTC
GAGGACTTCAAGGTTATATAAGGGGTGGATTGATGTATCTTCGAGAAGGGATTGAGTTGT
AGTTTCGTTTCCCAATTCTTACTTAAGTTGTTTTATTTTCTCTATTTGTAAGATAAGCAC
ATCAAAAGAAAAGTAATCAAGTATTACAAGAAACAAAAATTCAAGTAAATAACAGATAAT
ATGTCAAAAGCTGTCGGTATTGATTTAGGTACAACATACTCGTGTGTTGCTCACTTTGCT
AATGATCGTGTGGACATTATTGCCAACGATCAAGGTAACAGAACCACTCCATCTTTTGTC
GCTTTCACTGACACTGAAAGATTGATTGGTGATGCTGCTAAGAATCAAGCTGCTATGAAT
CCTTCGAATACCGTTTTCGACGCTAAGCGTTTGATCGGTAGAAACTTCAACGACCCAGAA
GTGCAGGCTGACATGAAGCACTTCCCATTCAAGTTGATCGATGTTGACGGTAAGCCTCAA
ATTCAAGTTGAATTTAAGGGTGAAACCAAGAACTTTACCCCAGAACAAATCTCCTCCATG
GTCTTGGGTAAGATGAAGGAAACTGCCGAATCTTACTTGGGAGCCAAGGTCAATGACGCT
GTCGTCACTGTCCCAGCTTACTTCAACGATTCTCAAAGACAAGCTACCAAGGATGCTGGT
ACCATTGCTGGTTTGAATGTCTTGCGTATTATTAACGAACCTACCGCCGCTGCCATTGCT
TACGGTTTGGACAAGAAGGGTAAGGAAGAACACGTCTTGATTTTCGACTTGGGTGGTGGT
ACTTTCGATGTCTCTTTGTTGTCCATTGAAGACGGTATCTTTGAAGTTAAGGCCACCGCT
GGTGACACCCATTTGGGTGGTGAAGATTTTGACAACAGATTGGTCAACCACTTCATCCAA
GAATTCAAGAGAAAGAACAAGAAGGACTTGTCTACCAACCAAAGAGCTTTGAGAAGATTA
AGAACCGCTTGTGAAAGAGCCAAGAGAACTTTGTCTTCCTCCGCTCAAACTTCCGTTGAA
ATTGACTCTTTGTTCGAAGGTATCGATTTCTACACTTCCATCACCAGAGCCAGATTCGAA
GAATTGTGTGCTGACTTGTTCAGATCTACTTTGGACCCAGTTGAAAAGGTCTTGAGAGAT
GCTAAATTGGACAAATCTCAAGTCGATGAAATTGTCTTGGTCGGTGGTTCTACCAGAATT
CCAAAGGTCCAAAAATTGGTCACTGACTACTTCAACGGTAAGGAACCAAACAGATCTATC
AACCCAGATGAAGCTGTTGCTTACGGTGCTGCTGTTCAAGCTGCTATTTTGACTGGTGAC
GAATCTTCCAAGACTCAAGATCTATTGTTGTTGGATGTCGCTCCATTATCCTTGGGTATT
GAAACTGCTGGTGGTGTCATGACCAAGTTGATTCCAAGAAACTCTACCATTCCAACAAAG
AAGTCCGAGATCTTTTCCACTTATGCTGATAACCAACCAGGTGTCTTGATTCAAGTCTTT
GAAGGTGAAAGAGCCAAGACTAAGGACAACAACTTGTTGGGTAAGTTCGAATTGAGTGGT
ATTCCACCAGCTCCAAGAGGTGTCCCACAAATTGAAGTCACTTTCGATGTCGACTCTAAC
GGTATTTTGAATGTTTCCGCCGTCGAAAAGGGTACTGGTAAGTCTAACAAGATCACTATT
ACCAACGACAAGGGTAGATTGTCCAAGGAAGATATCGAAAAGATGGTTGCTGAAGCCGAA
AAATTCAAGGAAGAAGATGAAAAGGAATCTCAAAGAATTGCTTCCAAGAACCAATTGGAA
TCCATTGCTTACTCTTTGAAGAACACCATTTCTGAAGCTGGTGACAAATTGGAACAAGCT
GACAAGGACACCGTCACCAAGAAGGCTGAAGAGACTATTTCTTGGTTAGACAGCAACACC
ACTGCCAGCAAGGAAGAATTCGATGACAAGTTGAAGGAGTTGCAAGACATTGCCAACCCA
ATCATGTCTAAGTTGTACCAAGCTGGTGGTGCTCCAGGTGGCGCTGCAGGTGGTGCTCCA
GGCGGTTTCCCAGGTGGTGCTCCTCCAGCTCCAGAGGCTGAAGGTCCAACCGTTGAAGAA
GTTGATTAAGCCAATTGGTGCGGCAATTGATAATAACGAAAATGTCTTTTAATGATCTGG
GTATAATGAGGAATTTTCCGAACGTTTTTACTTTATATATATATATACATGTAACATATA
TTCTATACGCTATAGAGAAAGGAAATTTTTCAATTAAAAAAAAAATAGAGAAAGAGTTTC
ACTTCTTGATTATCGCTAACACTAATGGTTGAAGTACTGCTACTTTAATTTTATAGATAG
GCAAAAAAAAATTATTCGGGGCGAGCTGGGAATTGAACCCAGGGCCTCTCGCATGCTTTG
TCTTCCTGTTTAATCAGGAAGTCGCCCAAAGCGAGAATCATACCACTAGACCACACGCCC
GTACTAATTGATGTCTTCCTTTTCGGATAGATGTATATATATACAAATTGGTCAGATTGC
TTTTGGCTCCCTTTCGTACGTAACTCATTTAGACTACGGATCACTAGCACTATCTCACCA
AGTTTTTAAAAGATCCACTGTGATCATTAAAGATTCTATTTCAAATAAAAATCAATTATC
ATCTATCGACTAGTTTTCATGGTACTAGTATATTATCATGTACAGTGTGAGGGCTCGACA
TGAAGATTG"""


In [48]:
dna = dna.replace("\n","").replace("\r","")
dna = dna[::-1]
dna=dna.replace("A",'t').replace('T','a').replace("C","g").replace('G',"c")

In [49]:
ind=dna.find("aaagtgact")
print ind

1105


This is the starting position for the kmer. Now we can extract 30,150 and 500 bp starting from ind, run blast here https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome and compare the results.

In [53]:
dna[ind:ind+30]

'aaagtgacttcaatttgtgggacacctctt'

Two identical sequences are found: chrI and chrXII

In [54]:
dna[ind:ind+150]

'aaagtgacttcaatttgtgggacacctcttggagctggtggaataccactcaattcgaacttacccaacaagttgttgtccttagtcttggctctttcaccttcaaagacttgaatcaagacacctggttggttatcagcataagtggaa'

There is only one entry in blast that is 100% identical (sequence itself at a position 140008) and another entry that finds similar sequence on chrXII, query cover 97%, identity 99%. If not take it into account then this read can be considered as uniquely mappable.

In [55]:
dna[ind:ind+500]

'aaagtgacttcaatttgtgggacacctcttggagctggtggaataccactcaattcgaacttacccaacaagttgttgtccttagtcttggctctttcaccttcaaagacttgaatcaagacacctggttggttatcagcataagtggaaaagatctcggacttctttgttggaatggtagagtttcttggaatcaacttggtcatgacaccaccagcagtttcaatacccaaggataatggagcgacatccaacaacaatagatcttgagtcttggaagattcgtcaccagtcaaaatagcagcttgaacagcagcaccgtaagcaacagcttcatctgggttgatagatctgtttggttccttaccgttgaagtagtcagtgaccaatttttggacctttggaattctggtagaaccaccgaccaagacaatttcatcgacttgagatttgtccaatttagcatctctcaagaccttttcaactgggtccaaagtaga'

This sequence was found two times in the genome, 2nd time is on chromosome XII with 98% identity. Since percent mismatches allowed by the GEM program is 4%, this sequence is included into the frequency and mappability scores.

In [1]:
import pandas as pd

In [2]:
table = pd.DataFrame(columns = ("position","kmer","GEM-mappability","Blast-mappability"))

In [13]:
table.loc[1]=140008, 30, 0.5, 0.5
table.loc[2]=140008, 150, 1, 1
table.loc[3]=140008, 500, 0.5, 0.5
table[["position","kmer"]]=table[["position","kmer"]].astype(int)
table

,position,kmer,GEM-mappability,Blast-mappability
1,140008,30,0.50,0.50
2,140008,150,1.00,1.00
3,140008,500,0.50,0.50
4,469317,30,0.25,0.25
5,469317,150,0.25,0.25
6,469317,500,0.25,0.25


Blast results are the same as the GEM-computed ones

#### Chromosome XII, position 469317  
 <img src="../images/yeastchrXII_YLR155C.png">



According to the image above the mappability at this position for all the kmers should be the same - 0.25. 

In [4]:
dna2 = """ATGAGATCTTTAAATACCCTTTTACTTTCTCTCTTTGTCGCAATGTCCAGTGGTGCTCCA
CTACTAAAAATTCGTGAAGAGAAGAATTCTTCTTTGCCATCAATCAAAATTTTTGGTACC
GGCGGTACTATCGCTTCCAAGGGTTCGACAAGTGCAACAACGGCGGGTTATAGCGTGGGA
TTAACCGTAAATGATTTAATAGAAGCCGTCCCATCTTTAGCTGAGAAGGCAAATCTGGAC
TATCTTCAAGTGTCTAACGTTGGTTCAAATTCTTTAAACTATACGCATCTGATCCCATTG
TATCACGGTATCTCCGAGGCACTAGCCTCTGATGACTACGCTGGTGCGGTTGTCACTCAT
GGGACCGACACTATGGAGGAGACAGCTTTCTTCTTAGATTTGACCATAAATTCAGAGAAG
CCAGTATGTATCGCAGGCGCTATGCGTCCAGCCACTGCCACGTCTGCTGATGGCCCAATG
AATTTATATCAAGCAGTGTCTATTGCTGCTTCTGAGAAATCACTGGGTCGTGGCACGATG
ATCACTCTAAACGATCGTATTGCCTCTGGGTTTTGGACAACGAAAATGAATGCCAACTCT
TTAGATACATTCAGAGCGGATGAACAGGGATATTTAGGTTACTTTTCAAATGATGACGTG
GAGTTTTACTACCCACCAGTCAAGCCAAATGGATGGCAATTTTTTGACATTTCCAACCTC
ACAGACCCTTCGGAAATTCCAGAAGTCATTATTCTGTACTCCTATCAAGGCTTGAATCCT
GAGCTAATAGTAAAGGCCGTCAAGGACCTGGGCGCAAAAGGTATCGTGTTGGCGGGTTCT
GGAGCTGGTTCCTGGACTGCTACGGGTAGTATTGTAAACGAACAACTTTATGAAGAGTAT
GGTATACCAATTGTTCACAGCAGAAGAACAGCAGATGGTACAGTTCCTCCAGATGATGCC
CCAGAGTACGCCATTGGATCTGGCTACCTAAACCCTCAAAAATCGCGTATTTTGCTACAA
TTATGTTTGTACTCCGGCTACGGCATGGATCAGATTAGGTCTGTTTTTTCTGGCGTCTAC
GGTGGTTAA"""

In [5]:
dna2 = dna2.replace("\n","").replace("\r","")
dna2 = dna2[::-1]
dna2 = dna2.replace("A",'t').replace('T','a').replace("C","g").replace('G',"c")
dna2

'ttaaccaccgtagacgccagaaaaaacagacctaatctgatccatgccgtagccggagtacaaacataattgtagcaaaatacgcgatttttgagggtttaggtagccagatccaatggcgtactctggggcatcatctggaggaactgtaccatctgctgttcttctgctgtgaacaattggtataccatactcttcataaagttgttcgtttacaatactacccgtagcagtccaggaaccagctccagaacccgccaacacgataccttttgcgcccaggtccttgacggcctttactattagctcaggattcaagccttgataggagtacagaataatgacttctggaatttccgaagggtctgtgaggttggaaatgtcaaaaaattgccatccatttggcttgactggtgggtagtaaaactccacgtcatcatttgaaaagtaacctaaatatccctgttcatccgctctgaatgtatctaaagagttggcattcattttcgttgtccaaaacccagaggcaatacgatcgtttagagtgatcatcgtgccacgacccagtgatttctcagaagcagcaatagacactgcttgatataaattcattgggccatcagcagacgtggcagtggctggacgcatagcgcctgcgatacatactggcttctctgaatttatggtcaaatctaagaagaaagctgtctcctccatagtgtcggtcccatgagtgacaaccgcaccagcgtagtcatcagaggctagtgcctcggagataccgtgatacaatgggatcagatgcgtatagtttaaagaatttgaaccaacgttagacacttgaagatagtccagatttgccttctcagctaaagatgggacggcttctattaaatcatttacggttaatcccacgctataacccgccgttgttgcacttgtcgaacccttggaagcgatagtaccgccggtaccaaaaattttgattgatggcaaaga

In [6]:
ind2=dna2.find("ttaaccacc")
print ind2

0


In [7]:
dna2[ind2:ind2+30]

'ttaaccaccgtagacgccagaaaaaacaga'

The expected mappability is 0.25, nucleotide blast finds 4 identical sequences within the chromosome XII. To find out positions of this sequences click on __TPA: Saccharomyces cerevisiae S288C chromosome XII, complete sequence__, all entries are listed in a position ascending order.

In [8]:
dna2[ind2:ind2+150]

'ttaaccaccgtagacgccagaaaaaacagacctaatctgatccatgccgtagccggagtacaaacataattgtagcaaaatacgcgatttttgagggtttaggtagccagatccaatggcgtactctggggcatcatctggaggaactgt'

In [9]:
dna2[ind2:ind2+500]

'ttaaccaccgtagacgccagaaaaaacagacctaatctgatccatgccgtagccggagtacaaacataattgtagcaaaatacgcgatttttgagggtttaggtagccagatccaatggcgtactctggggcatcatctggaggaactgtaccatctgctgttcttctgctgtgaacaattggtataccatactcttcataaagttgttcgtttacaatactacccgtagcagtccaggaaccagctccagaacccgccaacacgataccttttgcgcccaggtccttgacggcctttactattagctcaggattcaagccttgataggagtacagaataatgacttctggaatttccgaagggtctgtgaggttggaaatgtcaaaaaattgccatccatttggcttgactggtgggtagtaaaactccacgtcatcatttgaaaagtaacctaaatatccctgttcatccgctctgaatgtatctaaagagttggcat'

With blast we find 4 entries for each kmer which corresponds to the GEM-mappability of 0.25; the results seem to be correct.

In [14]:
table.loc[4]=469317, 30, 0.25, 0.25
table.loc[5]=469317, 150, 0.25, 0.25
table.loc[6]=469317, 500, 0.25, 0.25
table[["position","kmer"]]=table[["position","kmer"]].astype(int)
table

,position,kmer,GEM-mappability,Blast-mappability
1,140008,30,0.50,0.50
2,140008,150,1.00,1.00
3,140008,500,0.50,0.50
4,469317,30,0.25,0.25
5,469317,150,0.25,0.25
6,469317,500,0.25,0.25


## Mappability of the human chrY

<img src="../images/humanChrYMappability.png">

To find out positions of the gaps use UCSC table browser http://genome.ucsc.edu/cgi-bin/hgTables (__group__: mapping and sequencing, __track__: gap).  According to the table the biggest gap is heterochromatin spanning the region 26'673'214-56'673'214, which is 30'000'000 bp long. Centromere is at the position 10'316'944-10'544'039. This regions appear unmappable also in IGV.